# This notebook explores the initial scripts necessary to extract the financial data from Apple's SEC Filings

In [1]:
import pandas as pd

file_to_inspect = 'data/2024-10q.xls'

try:
    # First, we open the Excel file
    xls = pd.ExcelFile(file_to_inspect)
    
    # Now, we can ask it for the names of all the sheets inside
    print(f" Successfully opened {file_to_inspect}.")
    print("Here are the names of all the sheets found inside:")
    print(xls.sheet_names)

except Exception as e:
    print(f" An error occurred: {e}")

 Successfully opened data/2024-10q.xls.
Here are the names of all the sheets found inside:
['Table_Of_Contents', 'TABLE1', 'TABLE2', 'TABLE3', 'INCOME_STATEMENT', 'TABLE4', 'BALANCE_SHEET', 'BALANCE_SHEET2', 'STOCKHOLDERS_EQUITY', 'CASH_FLOW', 'TABLE5', 'TABLE6', 'TABLE7', 'TABLE8', 'TABLE9', 'TABLE10', 'TABLE11', 'TABLE12', 'TABLE13', 'TABLE14', 'TABLE15', 'TABLE16', 'TABLE17', 'TABLE18', 'TABLE19', 'TABLE20', 'TABLE21', 'TABLE22', 'TABLE23', 'TABLE24']


### Attempting to extract main data from the income statement of the 2024 10q filing

In [3]:
import pandas as pd
import os

# --- Part 1: SETUP FOR A SINGLE FILE TEST ---
file_to_test = 'data/2024-10q.xls' 

print(f"--- Starting Test on Single File: {file_to_test} ---")


# --- Part 2: THE FINAL, REFINED read_excel COMMAND ---
df = pd.read_excel(
    file_to_test, 
    sheet_name='INCOME_STATEMENT', # Correct sheet name
    skiprows=18, # CORRECTED: Skip 18 rows to start at "Net sales"
    header=None,
    usecols="B:F",
    names=[
        'Metric', 
        'Three Months Ended 2024',
        'Three Months Ended 2023',
        'Nine Months Ended 2024',
        'Nine Months Ended 2023'
    ]
)

# --- Part 3: VERIFY THE RESULT ---
print("\n Successfully read the data. Here's the first look:")
print(df.head(10))

print("\n--- DataFrame Info ---")
df.info()

--- Starting Test on Single File: data/2024-10q.xls ---

 Successfully read the data. Here's the first look:
                Metric  Three Months Ended 2024  Three Months Ended 2023  \
0           Net sales:                      NaN                      NaN   
1             Products                  61564.0                  60584.0   
2             Services                  24213.0                  21213.0   
3      Total net sales                  85777.0                  81797.0   
4                  NaN                      NaN                      NaN   
5       Cost of sales:                      NaN                      NaN   
6             Products                  39803.0                  39136.0   
7             Services                   6296.0                   6248.0   
8  Total cost of sales                  46099.0                  45384.0   
9         Gross margin                  39678.0                  36413.0   

   Nine Months Ended 2024  Nine Months Ended 2023  
0 

### Cleaning the data

In [4]:
import pandas as pd
import os

# --- Part 1: SETUP FOR A SINGLE FILE TEST ---
file_to_test = 'data/2024-10q.xls' 
print(f"--- Starting Test on Single File: {file_to_test} ---")


# --- Part 2: THE EXTRACTION COMMAND ---
df = pd.read_excel(
    file_to_test, 
    sheet_name='INCOME_STATEMENT',
    skiprows=18,
    header=None,
    usecols="B:F",
    names=[
        'Metric', 
        'Three Months Ended 2024',
        'Three Months Ended 2023',
        'Nine Months Ended 2024',
        'Nine Months Ended 2023'
    ]
)

# --- Part 3: THE CLEANING AND TRANSFORMATION ---
print("\n--- Starting Data Cleaning ---")

# Step A: Remove rows where all numeric columns are empty
numeric_cols = [
    'Three Months Ended 2024', 'Three Months Ended 2023', 
    'Nine Months Ended 2024', 'Nine Months Ended 2023'
]
df.dropna(subset=numeric_cols, how='all', inplace=True)

# Step B: Clean up the 'Metric' column text
# .str.strip() removes leading/trailing whitespace
# .str.replace(':', '') removes the colon character
df['Metric'] = df['Metric'].str.strip().str.replace(':', '')

# Step C: Filter the DataFrame to keep only the rows we care about
# gets rid of sub-headers like "Net sales" which have no data
metrics_to_keep = [
    'Products', # This is a component of Net sales
    'Services', # This is a component of Net sales
    'Total net sales',
    'Gross margin',
    'Research and development',
    'Selling, general and administrative',
    'Total operating expenses',
    'Operating income',
    'Net income',
    'Earnings per share, Basic',
    'Earnings per share, Diluted'
]
df = df[df['Metric'].isin(metrics_to_keep)]


# --- Part 4: VERIFY THE CLEANED RESULT ---
print("\n Successfully Cleaned the data. Here's the final look:")
print(df) # Print the whole thing now that it's smaller

print("\n--- Final DataFrame Info ---")
df.info()

--- Starting Test on Single File: data/2024-10q.xls ---

--- Starting Data Cleaning ---

 Successfully Cleaned the data. Here's the final look:
                                 Metric  Three Months Ended 2024  \
1                              Products                  61564.0   
2                              Services                  24213.0   
3                       Total net sales                  85777.0   
6                              Products                  39803.0   
7                              Services                   6296.0   
9                          Gross margin                  39678.0   
12             Research and development                   8006.0   
13  Selling, general and administrative                   6320.0   
14             Total operating expenses                  14326.0   
16                     Operating income                  25352.0   
20                           Net income                  21448.0   

    Three Months Ended 2023  Nine Month

### Melting data to prepare it for PowerBI

In [5]:
# --- Part 5: Reshape the Data for Visualization ---
# 'id_vars' are the columns we want to keep as they are.
# The other columns will be "unpivoted".
df_melted = df.melt(
    id_vars=['Metric'],
    var_name='Period', # The name for the new column holding the old column headers
    value_name='Amount' # The name for the new column holding the values
)

print("--- Data after unpivoting (melting) ---")
display(df_melted)

--- Data after unpivoting (melting) ---


,Metric,Period,Amount
0,Products,Three Months Ended 2024,61564.0
1,Services,Three Months Ended 2024,24213.0
2,Total net sales,Three Months Ended 2024,85777.0
3,Products,Three Months Ended 2024,39803.0
4,Services,Three Months Ended 2024,6296.0
5,Gross margin,Three Months Ended 2024,39678.0
6,Research and development,Three Months Ended 2024,8006.0
7,"Selling, general and administrative",Three Months Ended 2024,6320.0
8,Total operating expenses,Three Months Ended 2024,14326.0
9,Operating income,Three Months Ended 2024,25352.0


In [6]:
# --- Part 6: Feature Engineering ---
# Extract the Year from the 'Period' column using a regular expression
df_melted['Year'] = df_melted['Period'].str.extract(r'(\d{4})').astype(int)

# Extract the period type ('Three Months' or 'Nine Months')
df_melted['Period Type'] = df_melted['Period'].str.extract(r'^(Three|Nine)')[0]

print("--- Data after Feature Engineering ---")
display(df_melted.head())

--- Data after Feature Engineering ---


,Metric,Period,Amount,Year,Period Type
0,Products,Three Months Ended 2024,61564.0,2024,Three
1,Services,Three Months Ended 2024,24213.0,2024,Three
2,Total net sales,Three Months Ended 2024,85777.0,2024,Three
3,Products,Three Months Ended 2024,39803.0,2024,Three
4,Services,Three Months Ended 2024,6296.0,2024,Three
